# Machine Learning Approach

In [ ]:
# Local libraries
import Tools.ratings_utils as ru
import Tools.system_utils as sys

YEAR = 2024
FILENAME = f"Data/Seasons/data_{YEAR}.json"
TOURNAMENT_FILENAME = f"Data/Tournaments/tournament_{YEAR}.csv"
PICKS_FILENAME = f"Data/Tournament Picks/picks_{YEAR}.csv"


# Create data frame for valid teams in the current season that can be used for tournament simulation
score_df = ru.set_rating_data_frame(filename=FILENAME)

In [ ]:
print(score_df.head())

In [ ]:
rating_score_df = ru.add_ratings_per_game(score_df=score_df)

In [ ]:
print(rating_score_df)